No contexto do PyTorch, o ajuste de parâmetros (tuning de parâmetros) refere-se ao processo de encontrar a configuração ideal dos hiperparâmetros de um modelo de aprendizado de máquina. Os hiperparâmetros são valores que não são aprendidos diretamente pelo modelo durante o treinamento, mas são definidos antes do treinamento e afetam o desempenho e o comportamento do modelo.

O ajuste de parâmetros é uma etapa crucial no desenvolvimento de modelos de aprendizado de máquina, pois diferentes combinações de hiperparâmetros podem ter um impacto significativo no desempenho do modelo. Alguns exemplos comuns de hiperparâmetros incluem a taxa de aprendizado, o número de camadas ocultas em uma rede neural, o tamanho do lote (batch size), a taxa de regularização, entre outros.

O processo de ajuste de parâmetros geralmente envolve a definição de um espaço de busca para os hiperparâmetros, em seguida, a realização de experimentos com diferentes combinações de valores de hiperparâmetros nesse espaço. Esses experimentos podem ser executados em um conjunto de validação separado para avaliar o desempenho do modelo em relação a diferentes configurações de hiperparâmetros. Com base nos resultados desses experimentos, os hiperparâmetros podem ser ajustados para melhorar o desempenho do modelo.

Uma abordagem comum para realizar o ajuste de parâmetros é a busca em grade (grid search) ou busca aleatória (random search), onde diferentes combinações de valores de hiperparâmetros são testadas de forma sistemática ou aleatória, respectivamente. Além disso, também existem técnicas mais avançadas, como otimização bayesiana ou algoritmos genéticos, que podem ajudar a encontrar combinações ótimas de hiperparâmetros de maneira mais eficiente.

Em resumo, o ajuste de parâmetros no PyTorch é o processo de encontrar os melhores valores para os hiperparâmetros de um modelo de aprendizado de máquina, com o objetivo de melhorar seu desempenho e generalização.

### 

# Projeto 3: Classificação binária brest cancer com tuning dos parâmetros

In [1]:
!pip install skorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.7/193.7 kB 6.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import sklearn
import skorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import GridSearchCV
from skorch import NeuralNetBinaryClassifier

In [3]:
torch.__version__, skorch.__version__, sklearn.__version__

('2.0.0+cu118', '0.12.1', '1.2.2')

In [4]:
np.random.seed(123)
torch.manual_seed(123)

In [5]:
previsores = pd.read_csv('entradas_breast.csv')
classe = pd.read_csv('saidas_breast.csv') # convertendo do formato pandas para numpy para sklearn fzr a leitura

In [6]:
previsores = np.array(previsores, dtype = 'float32')
classe = np.array(classe, dtype = 'float32').squeeze(1)

In [7]:
previsores.shape

(569, 30)

In [8]:
classe.shape

(569,)

### Classe para estrutura da rede neural

In [9]:
class classificador_torch(nn.Module):
  def __init__(self, activation, neurons, initializer):
    super().__init__()
    # 30 -> 16 -> 16 -> 1
    self.dense0 = nn.Linear(30, neurons)
    initializer(self.dense0.weight)
    self.activation0 = activation
    self.dense1 = nn.Linear(neurons, neurons)
    initializer(self.dense1.weight)
    self.activation1 = activation
    self.dense2 = nn.Linear(neurons, 1)
    initializer(self.dense2.weight)
    # self.output = nn.Sigmoid() ** ATUALIZAÇÃO (ver detalhes no texto acima) **

  def forward(self, X):
    X = self.dense0(X)
    X = self.activation0(X)
    X = self.dense1(X)
    X = self.activation1(X)
    X = self.dense2(X)
    # X = self.output(X) ** ATUALIZAÇÃO (ver detalhes no texto acima) **
    return X

### Skorch

In [10]:
classificador_sklearn = NeuralNetBinaryClassifier(module=classificador_torch,
                                                  lr = 0.001,
                                                  optimizer__weight_decay = 0.0001,
                                                  train_split=False)

### Tunning dos parâmetros

In [13]:
params = {'batch_size': [10, 30],
          'max_epochs': [50, 100],
          'optimizer': [torch.optim.Adam, torch.optim.SGD],
          'criterion': [torch.nn.BCEWithLogitsLoss], #, torch.nn.HingeEmbeddingLoss], # ** ATUALIZAÇÃO **
          'module__activation': [F.relu, F.tanh],
          'module__neurons': [8, 16], 
          'module__initializer': [torch.nn.init.uniform]} # _, torch.nn.init.normal_]}

In [14]:
params

{'batch_size': [10, 30],
 'max_epochs': [50, 100],
 'optimizer': [torch.optim.adam.Adam, torch.optim.sgd.SGD],
 'criterion': [torch.nn.modules.loss.BCEWithLogitsLoss],
 'module__activation': [<function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
  <function torch.nn.functional.tanh(input)>],
 'module__neurons': [8, 16],
 'module__initializer': [<function torch.nn.init._make_deprecate.<locals>.deprecated_init(*args, **kwargs)>]}

In [15]:
grid_search = GridSearchCV(estimator=classificador_sklearn, param_grid=params,
                           scoring = 'accuracy', cv = 2)
grid_search = grid_search.fit(previsores, classe)

  epoch    train_loss     dur
-------  ------------  ------
      1    27299.8736  0.1851


<ipython-input-9-c3eb0d2bf22a>:6: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense0.weight)
<ipython-input-9-c3eb0d2bf22a>:9: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense1.weight)
<ipython-input-9-c3eb0d2bf22a>:12: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense2.weight)


      2    24343.2036  0.0459
      3    21607.9807  0.0398
      4    19135.1212  0.0418
      5    16931.4985  0.0409
      6    14986.2365  0.0394
      7    13263.7471  0.0449
      8    11731.9747  0.0400
      9    10367.9509  0.0467
     10     9150.4847  0.0411
     11     8060.7277  0.0399
     12     7080.4200  0.0408
     13     6192.2920  0.0400
     14     5382.8836  0.0403
     15     4640.1404  0.0440
     16     3952.5611  0.0407
     17     3309.9055  0.0384
     18     2702.8676  0.0421
     19     2122.0805  0.0395
     20     1558.6191  0.0397
     21     1003.4047  0.0405
     22      478.4396  0.0442
     23      138.4233  0.0403
     24       90.7293  0.0409
     25       88.3919  0.0387
     26       77.7053  0.0434
     27       70.4368  0.0412
     28       64.3089  0.0398
     29       58.6982  0.0401
     30       54.5391  0.0455
     31       52.0841  0.0422
     32       48.6803  0.0463
     33       46.1666  0.0386
     34       44.8264  0.0491
     35   

<ipython-input-9-c3eb0d2bf22a>:6: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense0.weight)
<ipython-input-9-c3eb0d2bf22a>:9: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense1.weight)
<ipython-input-9-c3eb0d2bf22a>:12: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense2.weight)


      5    18214.9025  0.0512
      6    16167.1386  0.0405
      7    14310.4449  0.0396
      8    12627.0771  0.0412
      9    11098.6769  0.0384
     10     9706.9951  0.0393
     11     8434.7966  0.0435
     12     7264.4413  0.0453
     13     6185.1344  0.0411
     14     5186.7851  0.0375
     15     4250.3334  0.0401
     16     3362.9745  0.0387
     17     2533.8013  0.0406
     18     1763.0071  0.0401
     19     1031.2719  0.0443
     20      462.3747  0.0399
     21      238.6098  0.0402
     22      214.4065  0.0414
     23      198.2909  0.0401
     24      181.4063  0.0402
     25      171.1932  0.0428
     26      161.6545  0.0450
     27      153.4939  0.0499
     28      146.7081  0.0473
     29      138.1020  0.0391
     30      128.5930  0.0408
     31      125.4269  0.0406
     32      118.2043  0.0384
     33      110.6343  0.0390
     34      101.7881  0.0395
     35       94.7601  0.0445
     36       85.2593  0.0390
     37       83.5712  0.0390
     38   

...

     11        3.7045  0.0221
     12        3.6806  0.0138
     13        3.6568  0.0169
     14        3.6329  0.0129
     15        3.6090  0.0130
     16        3.5852  0.0128
     17        3.5613  0.0113
     18        3.5374  0.0153
     19        3.5136  0.0122
     20        3.4897  0.0128
     21        3.4659  0.0168
     22        3.4420  0.0126
     23        3.4181  0.0199
     24        3.3943  0.0172
     25        3.3704  0.0117
     26        3.3466  0.0145
     27        3.3227  0.0147
     28        3.2989  0.0127
     29        3.2750  0.0163
     30        3.2512  0.0131
     31        3.2273  0.0150
     32        3.2035  0.0179
     33        3.1796  0.0131
     34        3.1558  0.0176
     35        3.1319  0.0147
     36        3.1081  0.0121
     37        3.0842  0.0130
     38        3.0604  0.0149
     39        3.0366  0.0125
     40        3.0127  0.0181
     41        2.9889  0.0111
     42        2.9651  0.0140
     43        2.9412  0.0153
     44   

<ipython-input-9-c3eb0d2bf22a>:6: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense0.weight)
<ipython-input-9-c3eb0d2bf22a>:9: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense1.weight)
<ipython-input-9-c3eb0d2bf22a>:12: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense2.weight)


     12        2.9778  0.0113
     13        2.9555  0.0153
     14        2.9332  0.0162
     15        2.9109  0.0148
     16        2.8886  0.0179
     17        2.8664  0.0144
     18        2.8441  0.0122
     19        2.8218  0.0130
     20        2.7995  0.0166
     21        2.7772  0.0108
     22        2.7550  0.0177
     23        2.7327  0.0167
     24        2.7104  0.0161
     25        2.6882  0.0167
     26        2.6659  0.0140
     27        2.6437  0.0118
     28        2.6214  0.0121
     29        2.5992  0.0216
     30        2.5770  0.0129
     31        2.5548  0.0153
     32        2.5326  0.0119
     33        2.5104  0.0172
     34        2.4882  0.0145
     35        2.4660  0.0148
     36        2.4438  0.0184
     37        2.4216  0.0119
     38        2.3995  0.0184
     39        2.3774  0.0165
     40        2.3552  0.0126
     41        2.3331  0.0168
     42        2.3110  0.0133
     43        2.2889  0.0124
     44        2.2668  0.0169
     45   

<ipython-input-9-c3eb0d2bf22a>:6: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense0.weight)
<ipython-input-9-c3eb0d2bf22a>:9: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense1.weight)
<ipython-input-9-c3eb0d2bf22a>:12: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  initializer(self.dense2.weight)


      6    50421.4231  0.0380
      7    45253.2452  0.0372
      8    40564.0448  0.0321
      9    36306.5410  0.0321
     10    32437.7287  0.0316
     11    28922.2148  0.0318
     12    25735.8369  0.0322
     13    22841.0558  0.0325
     14    20204.7019  0.0322
     15    17803.0770  0.0341
     16    15613.2270  0.0357
     17    13610.5641  0.0301
     18    11771.1620  0.0311
     19    10076.4286  0.0337
     20     8508.1092  0.0333
     21     7047.5943  0.0301
     22     5676.8558  0.0386
     23     4378.8280  0.0391
     24     3136.2214  0.0294
     25     1931.7888  0.0323
     26      750.3006  0.0462
     27      147.6414  0.0463
     28      107.6393  0.0403
     29       96.4330  0.0427
     30       92.9547  0.0471
     31       83.0662  0.0443
     32       76.6465  0.0446
     33       70.0438  0.0405
     34       63.3285  0.0405
     35       57.2976  0.0419
     36       51.7575  0.0414
     37       45.7397  0.0421
     38       40.1094  0.0399
     39   

In [16]:
melhores_parametros = grid_search.best_params_
melhor_precisao = grid_search.best_score_

In [17]:
melhores_parametros

{'batch_size': 30,
 'criterion': torch.nn.modules.loss.BCEWithLogitsLoss,
 'max_epochs': 100,
 'module__activation': <function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
 'module__initializer': <function torch.nn.init._make_deprecate.<locals>.deprecated_init(*args, **kwargs)>,
 'module__neurons': 16,
 'optimizer': torch.optim.adam.Adam}

In [18]:
melhor_precisao

0.7538299975290339